In [1]:
!git clone https://github.com/Hotsnown/seminaire-bordeaux-2022.git seminaire &> /dev/null
%pip install nbautoeval &> /dev/null
!git clone https://github.com/chinmusique/outcome-prediction.git &> /dev/null

SyntaxError: ignored

# Cleaning the outcome identification dataset

In my last post (NLP - Text Manipulation) I got into the topic of Natural Language Processing.

However, before we can start with Machine Learning algorithms some preprocessing steps are needed. I will introduce these in this and the following posts. Since this is a coherent post series and will build on each other I recommend to start with reading this post.

For this publication the dataset Amazon Unlocked Mobile from the statistic platform “Kaggle” was used. You can download it from my “GitHub Repository”.

## 2 Import the Libraries and the Data


If you are using the nltk library for the first time, you should import and download the following:



In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.


True

In [3]:
import pandas as pd
import numpy as np

import pickle as pk

import warnings
warnings.filterwarnings("ignore")


from bs4 import BeautifulSoup
import unicodedata
import re

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

from nltk.corpus import stopwords


from nltk.corpus import wordnet
from nltk import pos_tag
from nltk import ne_chunk

from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from nltk.probability import FreqDist
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [5]:
import json
import pandas as pd
from pathlib import Path

!git clone https://github.com/chinmusique/outcome-prediction.git

# creating list of json files with full paths
paths = Path(r'/content/outcome-prediction/data/manual_annotation').glob("*.json")

df_list = [] # empty list to store dataframes during the loop
for p in paths:
    data = json.load(open(p))
    data = data["annotations"]
    df_nested_list = pd.json_normalize(
        data,
        meta=['name','description','custom_fields'],)
    df_nested_list['file_name'] = p.name  # creating column to store file name: p.name
    df_list.append(df_nested_list) # dataframe to the list

df_raw = pd.concat(df_list, axis=0, ignore_index=True) # creating One Large Dataframe from all stored in the list
print(df_raw.columns)

df = df_raw.drop(columns=["role",	"class",	"index",	"contentIdx.start",	"contentIdx.end", "_id", "index"])
df.head()

Cloning into 'outcome-prediction'...
remote: Enumerating objects: 584, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 584 (delta 3), reused 0 (delta 0), pack-reused 562
Receiving objects: 100% (584/584), 5.20 MiB | 10.82 MiB/s, done.
Resolving deltas: 100% (11/11), done.
Index(['_id', 'role', 'content', 'class', 'index', 'contentIdx.start',
       'contentIdx.end', 'file_name'],
      dtype='object')


,content,file_name
0,We reverse the orders of the trial court denyi...,77129.json
1,We therefore reverse Defendant's conviction an...,77129.json
2,NaN,77129.json
3,"For the reasons discussed herein, we reverse ...",18357.json
4,"Accordingly, the cause is reversed and remande...",18357.json


However, we will only work with the following part of the data set:



Let’s take a closer look at the first set of reviews:



In [9]:
df = df[["content"]]

df['content'].iloc[0]


"We reverse the orders of the trial court denying Defendant's motion to dismiss the indictment."

In [10]:
df.dtypes


content    object
dtype: object

To be on the safe side, I convert the reviews as strings to be able to work with them correctly.



In [12]:
df['content'] = df['content'].astype(str)

### 3 Definition of required Functions


All functions are summarized here. I will show them again in the course of this post at the place where they are used.



In [14]:
def remove_html_tags_func(text):
    '''
    Removes HTML-Tags from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without HTML-Tags
    ''' 
    return BeautifulSoup(text, 'html.parser').get_text()

In [15]:
def remove_url_func(text):
    '''
    Removes URL addresses from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without URL addresses
    ''' 
    return re.sub(r'https?://\S+|www\.\S+', '', text)

In [16]:
def remove_accented_chars_func(text):
    '''
    Removes all accented characters from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without accented characters
    '''
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

In [17]:
def remove_punctuation_func(text):
    '''
    Removes all punctuation from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without punctuations
    '''
    return re.sub(r'[^a-zA-Z0-9]', ' ', text)

In [18]:
def remove_irr_char_func(text):
    '''
    Removes all irrelevant characters (numbers and punctuation) from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without irrelevant characters
    '''
    return re.sub(r'[^a-zA-Z]', ' ', text)

In [19]:
def remove_extra_whitespaces_func(text):
    '''
    Removes extra whitespaces from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without extra whitespaces
    ''' 
    return re.sub(r'^\s*|\s\s*', ' ', text).strip()

In [20]:
def word_count_func(text):
    '''
    Counts words within a string
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Number of words within a string, integer
    ''' 
    return len(text.split())

### 4 Text Pre-Processing


There are some text pre-processing steps to consider and a few more you can do. In this post I will talk about text cleaning.



### 4.1 Text Cleaning


Here I have created an example string, where you can understand the following steps very well.



In [21]:
messy_text = \
"Hi e-v-e-r-y-o-n-e !!!@@@!!! I gave a 5-star rating. \
Bought this special product here: https://www.amazon.com/. Another link: www.amazon.com/ \
Here the HTML-Tag as well:  <a href='https://www.amazon.com/'> …</a>. \
I HIGHLY RECOMMEND THIS PRDUCT !! \
I @ (love) [it] <for> {all} ~it's* |/ #special / ^^characters;! \
I am currently investigating the special device and am excited about the features. Love it! \
Furthermore, I found the support really great. Paid about 180$ for it (5.7inch version, 4.8'' screen). \
Sómě special Áccěntěd těxt and words like résumé, café or exposé.\
"
messy_text

"Hi e-v-e-r-y-o-n-e !!!@@@!!! I gave a 5-star rating. Bought this special product here: https://www.amazon.com/. Another link: www.amazon.com/ Here the HTML-Tag as well:  <a href='https://www.amazon.com/'> …</a>. I HIGHLY RECOMMEND THIS PRDUCT !! I @ (love) [it] <for> {all} ~it's* |/ #special / ^^characters;! I am currently investigating the special device and am excited about the features. Love it! Furthermore, I found the support really great. Paid about 180$ for it (5.7inch version, 4.8'' screen). Sómě special Áccěntěd těxt and words like résumé, café or exposé."

In the following I will perform the individual steps for text cleaning and always use parts of the messy_text string.



### 4.1.1 Conversion to Lower Case


In general, it is advisable to format the text completely in lower case.



In [22]:
messy_text_lower_case = \
"I HIGHLY RECOMMEND THIS PRDUCT !!\
"
messy_text_lower_case

'I HIGHLY RECOMMEND THIS PRDUCT !!'

In [23]:
messy_text_lower_case.lower()


'i highly recommend this prduct !!'

4.1.2 Removing HTML-Tags


In [24]:
messy_text_html = \
"Here the HTML-Tag as well:  <a href='https://www.amazon.com/'> …</a>.\
"
messy_text_html

"Here the HTML-Tag as well:  <a href='https://www.amazon.com/'> …</a>."

In [25]:
def remove_html_tags_func(text):
    '''
    Removes HTML-Tags from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without HTML-Tags
    ''' 
    return BeautifulSoup(text, 'html.parser').get_text()

In [26]:
remove_html_tags_func(messy_text_html)


'Here the HTML-Tag as well:   ….'

4.1.3 Removing URLs


In [27]:
messy_text_url = \
"Bought this product here: https://www.amazon.com/. Another link: www.amazon.com/\
"
messy_text_url

'Bought this product here: https://www.amazon.com/. Another link: www.amazon.com/'

In [28]:
def remove_url_func(text):
    '''
    Removes URL addresses from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without URL addresses
    ''' 
    return re.sub(r'https?://\S+|www\.\S+', '', text)

In [29]:
remove_url_func(messy_text_url)


'Bought this product here:  Another link: '

4.1.4 Removing Accented Characters


In [30]:
messy_text_accented_chars = \
"Sómě Áccěntěd těxt and words like résumé, café or exposé.\
"
messy_text_accented_chars

'Sómě Áccěntěd těxt and words like résumé, café or exposé.'

In [31]:
def remove_accented_chars_func(text):
    '''
    Removes all accented characters from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without accented characters
    '''
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

In [32]:
remove_accented_chars_func(messy_text_accented_chars)


'Some Accented text and words like resume, cafe or expose.'

4.1.5 Removing Punctuation


Punctuation is essentially the following set of symbols: [!”#$%&’()*+,-./:;<=>?@[]^_`{|}~]

In [33]:
messy_text_remove_punctuation = \
"Furthermore, I found the support really great. Paid about 180$ for it (5.7inch version, 4.8'' screen).\
"
messy_text_remove_punctuation

"Furthermore, I found the support really great. Paid about 180$ for it (5.7inch version, 4.8'' screen)."

In [34]:
def remove_punctuation_func(text):
    '''
    Removes all punctuation from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without punctuations
    '''
    return re.sub(r'[^a-zA-Z0-9]', ' ', text)

In [35]:
remove_punctuation_func(messy_text_remove_punctuation)


'Furthermore  I found the support really great  Paid about 180  for it  5 7inch version  4 8   screen  '

4.1.6 Removing irrelevant Characters (Numbers and Punctuation)


In [36]:
messy_text_irr_char = \
"Furthermore, I found the support really great. Paid about 180$ for it (5.7inch version, 4.8'' screen).\
"
messy_text_irr_char

"Furthermore, I found the support really great. Paid about 180$ for it (5.7inch version, 4.8'' screen)."

I am aware that this is the same example sentence as in the previous example, but here the difference between this and the previous function is made clear.



In [37]:
def remove_irr_char_func(text):
    '''
    Removes all irrelevant characters (numbers and punctuation) from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without irrelevant characters
    '''
    return re.sub(r'[^a-zA-Z]', ' ', text)

In [38]:
remove_irr_char_func(messy_text_irr_char)


'Furthermore  I found the support really great  Paid about      for it     inch version        screen  '

4.1.7 Removing extra Whitespaces


In [39]:
messy_text_extra_whitespaces = \
"I  am   a  text    with  many   whitespaces.\
"
messy_text_extra_whitespaces

'I  am   a  text    with  many   whitespaces.'

In [40]:
def remove_extra_whitespaces_func(text):
    '''
    Removes extra whitespaces from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without extra whitespaces
    ''' 
    return re.sub(r'^\s*|\s\s*', ' ', text).strip()

In [41]:
remove_extra_whitespaces_func(messy_text_extra_whitespaces)

'I am a text with many whitespaces.'

I always like to use this function in between, for example, when you have removed stop words, certain words or individual characters from the string(s). From time to time, this creates new whitespaces that I always like to remove for the sake of order.

4.1.8 Extra: Count Words


It is worthwhile to display the number of existing words, especially for validation of the pre-proessing steps. We will use this function again and again in later steps.



In [42]:
messy_text_word_count = \
"How many words do you think I will contain?\
"
messy_text_word_count

'How many words do you think I will contain?'

In [43]:
def word_count_func(text):
    '''
    Counts words within a string
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Number of words within a string, integer
    ''' 
    return len(text.split())

In [44]:
word_count_func(messy_text_word_count)

9

4.1.9 Extra: Expanding Contractions


You can do expanding contractions but you don’t have to. For the sake of completeness, I list the necessary functions, but do not use them in our following example with the Example String and DataFrame. I will give the reason for this in a later chapter.



In [45]:
from contractions import CONTRACTION_MAP 
import re 

def expand_contractions(text, map=CONTRACTION_MAP):
    pattern = re.compile('({})'.format('|'.join(map.keys())), flags=re.IGNORECASE|re.DOTALL)
    def get_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded = map.get(match) if map.get(match) else map.get(match.lower())
        expanded = first_char+expanded[1:]
        return expanded     
    new_text = pattern.sub(get_match, text)
    new_text = re.sub("'", "", new_text)
    return new_text

ModuleNotFoundError: ignored

With the help of this function, this sentence:



becomes the following:



This function should also work for this:



In [46]:
def word_count_func(text):
    '''
    Counts words within a string
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Number of words within a string, integer
    ''' 
    return len(text.split())

In [47]:
from pycontractions import Contractions
cont = Contractions(kv_model=model)
cont.load_models()# 

def expand_contractions(text):
    text = list(cont.expand_texts([text], precise=True))[0]
    return text

ModuleNotFoundError: ignored

4.1.10 Application to the Example String


Before that, I used individual text modules to show how all the text cleaning steps work. Now it is time to apply these functions to Example String (and subsequently to the DataFrame) one after the other.



In [48]:
messy_text


"Hi e-v-e-r-y-o-n-e !!!@@@!!! I gave a 5-star rating. Bought this special product here: https://www.amazon.com/. Another link: www.amazon.com/ Here the HTML-Tag as well:  <a href='https://www.amazon.com/'> …</a>. I HIGHLY RECOMMEND THIS PRDUCT !! I @ (love) [it] <for> {all} ~it's* |/ #special / ^^characters;! I am currently investigating the special device and am excited about the features. Love it! Furthermore, I found the support really great. Paid about 180$ for it (5.7inch version, 4.8'' screen). Sómě special Áccěntěd těxt and words like résumé, café or exposé."

In [49]:
messy_text_lower = messy_text.lower()
messy_text_lower

"hi e-v-e-r-y-o-n-e !!!@@@!!! i gave a 5-star rating. bought this special product here: https://www.amazon.com/. another link: www.amazon.com/ here the html-tag as well:  <a href='https://www.amazon.com/'> …</a>. i highly recommend this prduct !! i @ (love) [it] <for> {all} ~it's* |/ #special / ^^characters;! i am currently investigating the special device and am excited about the features. love it! furthermore, i found the support really great. paid about 180$ for it (5.7inch version, 4.8'' screen). sómě special áccěntěd těxt and words like résumé, café or exposé."

In [50]:
messy_text_wo_html = remove_html_tags_func(messy_text_lower)
messy_text_wo_html

"hi e-v-e-r-y-o-n-e !!!@@@!!! i gave a 5-star rating. bought this special product here: https://www.amazon.com/. another link: www.amazon.com/ here the html-tag as well:   …. i highly recommend this prduct !! i @ (love) [it]  {all} ~it's* |/ #special / ^^characters;! i am currently investigating the special device and am excited about the features. love it! furthermore, i found the support really great. paid about 180$ for it (5.7inch version, 4.8'' screen). sómě special áccěntěd těxt and words like résumé, café or exposé."

In [51]:
messy_text_wo_url = remove_url_func(messy_text_wo_html)
messy_text_wo_url

"hi e-v-e-r-y-o-n-e !!!@@@!!! i gave a 5-star rating. bought this special product here:  another link:  here the html-tag as well:   …. i highly recommend this prduct !! i @ (love) [it]  {all} ~it's* |/ #special / ^^characters;! i am currently investigating the special device and am excited about the features. love it! furthermore, i found the support really great. paid about 180$ for it (5.7inch version, 4.8'' screen). sómě special áccěntěd těxt and words like résumé, café or exposé."

In [52]:
messy_text_wo_acc_chars = remove_accented_chars_func(messy_text_wo_url)
messy_text_wo_acc_chars

"hi e-v-e-r-y-o-n-e !!!@@@!!! i gave a 5-star rating. bought this special product here:  another link:  here the html-tag as well:   .... i highly recommend this prduct !! i @ (love) [it]  {all} ~it's* |/ #special / ^^characters;! i am currently investigating the special device and am excited about the features. love it! furthermore, i found the support really great. paid about 180$ for it (5.7inch version, 4.8'' screen). some special accented text and words like resume, cafe or expose."

In [53]:
messy_text_wo_punct = remove_punctuation_func(messy_text_wo_acc_chars)
messy_text_wo_punct

'hi e v e r y o n e           i gave a 5 star rating  bought this special product here   another link   here the html tag as well         i highly recommend this prduct    i    love   it    all   it s      special     characters   i am currently investigating the special device and am excited about the features  love it  furthermore  i found the support really great  paid about 180  for it  5 7inch version  4 8   screen   some special accented text and words like resume  cafe or expose '

In [54]:
messy_text_wo_irr_char = remove_irr_char_func(messy_text_wo_punct)
messy_text_wo_irr_char

'hi e v e r y o n e           i gave a   star rating  bought this special product here   another link   here the html tag as well         i highly recommend this prduct    i    love   it    all   it s      special     characters   i am currently investigating the special device and am excited about the features  love it  furthermore  i found the support really great  paid about      for it     inch version        screen   some special accented text and words like resume  cafe or expose '

In [55]:
clean_text = remove_extra_whitespaces_func(messy_text_wo_irr_char)
clean_text

'hi e v e r y o n e i gave a star rating bought this special product here another link here the html tag as well i highly recommend this prduct i love it all it s special characters i am currently investigating the special device and am excited about the features love it furthermore i found the support really great paid about for it inch version screen some special accented text and words like resume cafe or expose'

In [56]:
print('Number of words: ' + str(word_count_func(clean_text)))


Number of words: 80


4.1.11 Application to the DataFrame


Now we apply the Text Cleaning Steps shown above to the DataFrame:



In [57]:
df.head()


,content
0,We reverse the orders of the trial court denyi...
1,We therefore reverse Defendant's conviction an...
2,nan
3,"For the reasons discussed herein, we reverse ..."
4,"Accordingly, the cause is reversed and remande..."


In [59]:
df['clean_content'] = df['content'].str.lower()
df['clean_content'] = df['clean_content'].apply(remove_html_tags_func)
df['clean_content'] = df['clean_content'].apply(remove_url_func)
df['clean_content'] = df['clean_content'].apply(remove_accented_chars_func)
df['clean_content'] = df['clean_content'].apply(remove_punctuation_func)
df['clean_content'] = df['clean_content'].apply(remove_irr_char_func)
df['clean_content'] = df['clean_content'].apply(remove_extra_whitespaces_func)

df.head()

,content,clean_content
0,We reverse the orders of the trial court denyi...,we reverse the orders of the trial court denyi...
1,We therefore reverse Defendant's conviction an...,we therefore reverse defendant s conviction an...
2,nan,nan
3,"For the reasons discussed herein, we reverse ...",for the reasons discussed herein we reverse an...
4,"Accordingly, the cause is reversed and remande...",accordingly the cause is reversed and remanded...


Let’s now compare the sentences from line 1 with the ones we have now edited:



In [63]:
df['content'].iloc[0]

"We reverse the orders of the trial court denying Defendant's motion to dismiss the indictment."

In [62]:
df['clean_content'].iloc[0]


'we reverse the orders of the trial court denying defendant s motion to dismiss the indictment'

Finally, we output the number of words and store them in a separate column. In this way, we can see whether and to what extent the number of words has changed in further steps.



In [66]:
df['Word_Count'] = df['clean_content'].apply(word_count_func)

df[['clean_content', 'Word_Count']].head()

,clean_content,Word_Count
0,we reverse the orders of the trial court denyi...,16
1,we therefore reverse defendant s conviction an...,15
2,nan,1
3,for the reasons discussed herein we reverse an...,12
4,accordingly the cause is reversed and remanded...,14


Here is the average number of words:



In [67]:
print('Average of words counted: ' + str(df['Word_Count'].mean()))


Average of words counted: 15.227422907488986


1. Jour 1
    * Variables
        * [exercice1](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/1.%20variables/1.1%20helloworld.ipynb)
        * [exercice2](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/1.%20variables/1.2%20%C3%A9viter%20les%20errreurs%20de%20nommage.ipynb)
        * [mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/1.%20variables/1.3%20mini-project.ipynb#scrollTo=RPB2xCMdA6lV)
    * Strings
        * [exercice1](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/2.%20strings/2.1%20concat.ipynb)
        * [exercice2](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/2.%20strings/2.2%20string_methods.ipynb)
        * [mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/2.%20strings/2.3%20mini-project.ipynb)
    * Opérations
        * [exercice1](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/3.%20operations/3.1%20math.ipynb)
        * [exercice2](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/3.%20operations/3.2%20bool%C3%A9en.ipynb)
        * [mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/3.%20operations/3.3%20mini-project.ipynb)

2. Jour 2
    * Listes
        * [Définition](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.1%20Listes/2.1.1%20d%C3%A9finition%20liste.ipynb?hl=fr)
        * [Strings et listes](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.1%20Listes/2.1.2%20String%20as%20list%20of%20characters.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.1%20Listes/2.1.3%20mini-project.ipynb?hl=fr)
    * Fonctions
        * [Définition I](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.2%20Fonctions/2.2.1%20d%C3%A9finition%20fonctions.ipynb?hl=fr)
        * [Définition II](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.2%20Fonctions/2.2.2%20scope%20et%20fonctions%20imbriqu%C3%A9es.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.2%20Fonctions/2.2.3%20mini-project.ipynb?hl=fr) 
    * Librairies
        * [Pandas](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.3%20Librairies/2.3.1%20Pandas.ipynb?hl=fr)
        * [Matplotlib](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.3%20Librairies/2.3.2%20Matplotlib.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.3%20Librairies/2.3.3%20mini-project.ipynb?hl=fr) 
3. Jour 3
    * Introduction à la NLP
        * [Charger des un corpus](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.1%20what%20is%20nlp/3.1.1%20Accessing%20Text.ipynb?hl=fr)
        * [Traitement de texte dans Pandas](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.1%20what%20is%20nlp/3.1.2%20Working%20with%20text%20data%20in%20pandas.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.1%20what%20is%20nlp/3.1.3%20mini-project.ipynb?hl=fr)
    * Segmentation
        * [Segmentation de tokens](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.2%20Segmentation/3.2.1%20Token%20segmentation.ipynb?hl=fr)
        * [Segmentation de phrase](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.2%20Segmentation/3.2.2%20Sentence%20segmentation.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.2%20Segmentation/3.2.3%20mini-project.ipynb?hl=fr)
    * Nettoyage de texte
        * [Stopwords](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.3%20text%20cleaning.ipynb/3.3.1%20stopwords.ipynb?hl=fr)
        * [Normalisation](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.3%20text%20cleaning.ipynb/3.3.2%20Normalizing%20Text.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.3%20text%20cleaning.ipynb/3.3.3%20mini-project.ipynb?hl=fr)
4. Jour 4
    * Apprentissage supervisé
        * [Régression linéaire](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.1%20supervised%20learning/4.3.1%20linear%20regression.ipynb?hl=fr)
        * [Evaluation](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.1%20supervised%20learning/4.3.2%20evaluale.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.1%20supervised%20learning/4.3.3%20mini-project.ipynb?hl=fr)
    * Pré-traitement de texte
        * [Featurization de textes](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.2%20text%20preprocessing/4.2.1%20text%20featurization.ipynb?hl=fr)
        * [Featurization de labels](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.2%20text%20preprocessing/4.2.2%20label%20featurization.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.2%20text%20preprocessing/4.2.3%20mini-project.ipynb?hl=fr)
    * Classification de texte
        * [EDA](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.3%20text%20classification/4.1.1%20EDA.ipynb?hl=fr)
        * [Apprentissage supervisé textuel](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.3%20text%20classification/4.1.1%20EDA.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.3%20text%20classification/4.1.3%20mini-project.ipynb?hl=fr)
5. Jour 5
    * [Projet final](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour5/final-project.ipynb?hl=fr)